In [1]:
import datetime
import os
import pickle

from dateutil.relativedelta import relativedelta

import camelot
import dateparser
import magic
import pandas as pd
import requests
import tabula

from bs4 import BeautifulSoup

In [2]:
!pwd

/home/sean/Projects/housing/housing/notebooks


In [14]:
os.chdir('/home/sean/Projects/housing/housing/data/raw')

In [19]:
pdfs = !ls
pdfs

['RAENewsRelease2-Feb2019.pdf',
 'RAENewsRelease-Apr2018.pdf',
 'RAENewsRelease-Apr2019.pdf',
 'RAENewsRelease-Aug2018.pdf',
 'RAENewsRelease-Dec2018.pdf',
 'RAENewsRelease-Feb2018.pdf',
 'RAENewsRelease-Jan2018.pdf',
 'RAENewsRelease-Jan2019.pdf',
 'RAENewsRelease-July2018.pdf',
 'RAENewsRelease-June2018.pdf',
 'RAENewsRelease-Mar2018.pdf',
 'RAENewsRelease-Mar2019.pdf',
 'RAENewsRelease-May2018.pdf',
 'RAENewsRelease-May2019.pdf',
 'RAENewsRelease-Nov2018.pdf',
 'RAENewsRelease-Oct2018.pdf',
 'RAENewsRelease-Sept2018.pdf',
 'YEGmedia-RAEstatsMay2019.pdf']

In [46]:
dates = {dateparser.parse(pdf.split('-')[1][:-4] if len(pdf.split('-')[1][:-4]) < 9 else "June2019",
                          settings={'PREFER_DAY_OF_MONTH': 'last'}) : pdf # pdf.split('-')[1][8:-4]
         for pdf in pdfs}
dates_sorted = {date - relativedelta(months=1):dates[date] for date in sorted(dates)}
dates_sorted

{datetime.datetime(2017, 12, 31, 0, 0): 'RAENewsRelease-Jan2018.pdf',
 datetime.datetime(2018, 1, 28, 0, 0): 'RAENewsRelease-Feb2018.pdf',
 datetime.datetime(2018, 2, 28, 0, 0): 'RAENewsRelease-Mar2018.pdf',
 datetime.datetime(2018, 3, 30, 0, 0): 'RAENewsRelease-Apr2018.pdf',
 datetime.datetime(2018, 4, 30, 0, 0): 'RAENewsRelease-May2018.pdf',
 datetime.datetime(2018, 5, 30, 0, 0): 'RAENewsRelease-June2018.pdf',
 datetime.datetime(2018, 6, 30, 0, 0): 'RAENewsRelease-July2018.pdf',
 datetime.datetime(2018, 7, 31, 0, 0): 'RAENewsRelease-Aug2018.pdf',
 datetime.datetime(2018, 8, 30, 0, 0): 'RAENewsRelease-Sept2018.pdf',
 datetime.datetime(2018, 9, 30, 0, 0): 'RAENewsRelease-Oct2018.pdf',
 datetime.datetime(2018, 10, 30, 0, 0): 'RAENewsRelease-Nov2018.pdf',
 datetime.datetime(2018, 11, 30, 0, 0): 'RAENewsRelease-Dec2018.pdf',
 datetime.datetime(2018, 12, 31, 0, 0): 'RAENewsRelease-Jan2019.pdf',
 datetime.datetime(2019, 1, 28, 0, 0): 'RAENewsRelease2-Feb2019.pdf',
 datetime.datetime(2019, 2

In [81]:
pdf = 'YEGmedia-RAEstatsMay2019.pdf'

In [52]:
# camelot didn't find any tables
# tables = camelot.read_pdf(pdf, pages='3-13', multiple_tables=True)

# tables

<TableList n=0>

In [49]:
def split_rows(df):
    new_rows = pd.DataFrame()
    for index, row in df.iterrows():
        #print(row[1])
        if pd.isnull(row[1]):
            #0print(row[0])
            pass
        try:
            if '/' in row[0]:
                split = row.str.split(pat='/',expand=True).transpose()
                split[0][1] = split[0][0].strip() + ' '+ split[0][1].strip()
                #print(split[0][1])
                new_rows = new_rows.append(split)
            else:
                new_rows = new_rows.append(row)
                #print(row[0])
        except:
            pass
    return new_rows.reset_index(drop=True)

In [91]:
df = tabula.read_pdf(pdf, pages='3-13', multiple_tables=True)

In [92]:
table = df[0]
split = split_rows(table)
split.head()

,0,1,2,3,4,5
0,gle Family Detached,NaN,NaN,NaN,NaN,NaN
1,New Listings,"2,281","2,526","2,137","1,932","2,083"
2,New Listings YTD,"9,029","9,479","8,319","8,785","9,082"
3,Sales,"1,244","1,212","1,250","1,322","1,239"
4,Sales YTD,"4,100","4,337","4,405","4,381","4,510"


In [93]:
index1 = ['Sigle Family Detached',
          'Condominium',
          'Duplex/Rowhouse',
          'Total Residential',
         ]
index2 = split.dropna()[0].drop_duplicates()
iterables = [index1, index2]
multi = pd.MultiIndex.from_product(iterables)
multi

MultiIndex(levels=[['Condominium', 'Duplex/Rowhouse', 'Sigle Family Detached', 'Total Residential'], ['Average Days on Market ', 'Average Days on Market YTD', 'Average Sale Price', 'Average Sale Price YTD', 'Median Days on Market ', 'Median Days on Market YTD', 'Median Sale Price', 'Median Sale Price YTD', 'New Listings ', 'New Listings YTD', 'Sales ', 'Sales Volume', 'Sales Volume YTD', 'Sales YTD', 'Sales to New Listings Ratio ', 'Sales to New Listings Ratio YTD']],
           codes=[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [8, 9, 10, 13, 14, 15, 11, 12, 2, 3, 6, 7, 0, 1, 4, 5, 8, 9, 10, 13, 14, 15, 11, 12, 2, 3, 6, 7, 0, 1, 4, 5, 8, 9, 10, 13, 14, 15, 11, 12, 2, 3, 6, 7, 0, 1, 4, 5, 8, 9, 10, 13, 14, 15, 11, 12, 2, 3, 6, 7, 0, 1, 4, 5]])

In [255]:
indexed = split.dropna().set_index(multi).drop(0, axis=1)
indexed.columns = range(2018,2013,-1)
indexed.head()

2018    2017    2016  \
Sigle Family Detached New Listings                  2,281   2,526   2,137    
                      New Listings YTD               9,029   9,479   8,319   
                      Sales                         1,244   1,212   1,250    
                      Sales YTD                      4,100   4,337   4,405   
                      Sales to New Listings Ratio     55%     48%     58%    

                                                      2015    2014  
Sigle Family Detached New Listings                  1,932   2,083   
                      New Listings YTD               8,785   9,082  
                      Sales                         1,322   1,239   
                      Sales YTD                      4,381   4,510  
                      Sales to New Listings Ratio     68%     59%

In [256]:
condo_df = indexed.iloc[indexed.index.get_level_values(0) == 'Condominium']
condo_df.head()

2018    2017    2016    2015  \
Condominium New Listings                  1,014   1,289   1,338   1,082    
            New Listings YTD               4,284   5,242   5,196   4,843   
            Sales                           472     474     470     474    
            Sales YTD                      1,656   1,799   1,849   1,808   
            Sales to New Listings Ratio     47%     37%     35%     44%    

                                            2014  
Condominium New Listings                  1,082   
            New Listings YTD               4,647  
            Sales                           553   
            Sales YTD                      2,034  
            Sales to New Listings Ratio     51%

In [201]:
condo_df_trans = condo_df.transpose()
condo_df_trans.index = pd.to_datetime(condo_df_trans.index, format='%Y')
condo_df_trans.index += pd.DateOffset(months=4)
condo_df_trans

Condominium                                    \
           New Listings  New Listings YTD Sales  Sales YTD   
2018-05-01        1,014             4,284   472      1,656   
2017-05-01        1,289             5,242   474      1,799   
2016-05-01        1,338             5,196   470      1,849   
2015-05-01        1,082             4,843   474      1,808   
2014-05-01        1,082             4,647   553      2,034   

                                                                         \
           Sales to New Listings Ratio  Sales to New Listings Ratio YTD   
2018-05-01                         47%                              39%   
2017-05-01                         37%                              34%   
2016-05-01                         35%                              36%   
2015-05-01                         44%                              37%   
2014-05-01                         51%                              44%   

                                                             \
           Sales Volume Sales Volume YTD Average Sale Price   
2018-05-01  112,242,686      374,606,797            237,802   
2017-05-01  116,199,037      433,921,822            245,146   
2016-05-01  119,512,573      461,864,180            254,282   
2015-05-01  121,350,417      454,041,593            256,014   
2014-05-01  144,273,597      520,517,655            260,893   

                                                                           \
           Average Sale Price YTD Median Sale Price Median Sale Price YTD   
2018-05-01                226,212           224,125               213,000   
2017-05-01                241,202           225,000               220,000   
2016-05-01                249,791           235,250               230,000   
2015-05-01                251,129           236,432               232,950   
2014-05-01                255,908           240,500               238,750   

                                                               \
           Average Days on Market  Average Days on Market YTD   
2018-05-01                     69                          75   
2017-05-01                     65                          72   
2016-05-01                     61                          67   
2015-05-01                     59                          61   
2014-05-01                     50                          50   

                                                             
           Median Days on Market  Median Days on Market YTD  
2018-05-01                    48                         55  
2017-05-01                    50                         54  
2016-05-01                    51                         49  
2015-05-01                    45                         46  
2014-05-01                    37                         37

In [50]:
def convert_pdf_to_df(pdf, pages='3-13'):
    df = tabula.read_pdf(pdf, pages=pages, multiple_tables=True)
    table = df[0]
    split = split_rows(table)
    index1 = ['Sigle Family Detached',
              'Condominium',
              'Duplex/Rowhouse',
              'Total Residential',
             ]
    index2 = split.dropna()[0].drop_duplicates()
    indices = [index1, index2]
    multi_index = pd.MultiIndex.from_product(indices)
    return split.dropna().set_index(multi_index).drop(0, axis=1)

indexed = convert_pdf_to_df(pdf)
indexed.head()

#indexed.columns = range(2018,2013,-1)
#condo_df = indexed.iloc[indexed.index.get_level_values(0) == 'Condominium']




1        2        3  \
Sigle Family Detached Listings                     646      515      719    
                      Listings YTD               17,428   17,109   18,425   
                      Sales                        554      451      464    
                      Sales YTD                  10,098    9,983   10,533   
                      Sales to Listings Ratio      86%      88%      65%    

                                                      4        5  
Sigle Family Detached Listings                     534      415   
                      Listings YTD               16,114   14,977  
                      Sales                        495      480   
                      Sales YTD                  11,536   10,513  
                      Sales to Listings Ratio      93%     116%

In [83]:
dfs = dict()
for date, pdf in dates_sorted.items():
    if date not in dfs.keys() and 'PDF' in magic.from_file(pdf):
        try:
            dfs[date] = convert_pdf_to_df(pdf)
        except Exception as e:
            print(e)

KeyError: 0

In [84]:
len(dfs)

18

In [71]:
pdf = 'RAENewsRelease-Apr2019.pdf'

In [82]:
magic.from_file(pdf)

'PDF document, version 1.7'

In [99]:
pd.concat(dfs.values(), axis=1)

1  \
Condominium           Average Days on Market                     75    
                      Average Days on Market YTD                  67   
                      Average Sale Price                     236,261   
                      Average Sale Price YTD                 248,130   
                      Listings                                  423    
                      Listings YTD                            10,455   
                      Median Days on Market                      63    
                      Median Days on Market YTD                   53   
                      Median Sale Price                      210,000   
                      Median Sale Price YTD                  227,000   
                      New Listings                               NaN   
                      New Listings YTD                           NaN   
                      Sales                                     247    
                      Sales Volume                        58,356,450   
                      Sales Volume YTD                 1,061,003,575   
                      Sales YTD                                4,276   
                      Sales to Listings Ratio                   58%    
                      Sales to Listings Ratio YTD                41%   
                      Sales to New Listings Ratio                NaN   
                      Sales to New Listings Ratio YTD            NaN   
Duplex/Rowhouse       Average Days on Market                     69    
                      Average Days on Market YTD                  63   
                      Average Sale Price                     337,020   
                      Average Sale Price YTD                 348,225   
                      Listings                                  134    
                      Listings YTD                             2,872   
                      Median Days on Market                      60    
                      Median Days on Market YTD                   49   
                      Median Sale Price                      325,000   
                      Median Sale Price YTD                  331,500   
...                                                              ...   
Sigle Family Detached New Listings                               NaN   
                      New Listings YTD                           NaN   
                      Sales                                     554    
                      Sales Volume                       230,844,885   
                      Sales Volume YTD                 4,420,333,993   
                      Sales YTD                               10,098   
                      Sales to Listings Ratio                   86%    
                      Sales to Listings Ratio YTD                58%   
                      Sales to New Listings Ratio                NaN   
                      Sales to New Listings Ratio YTD            NaN   
Total Residential     Average Days on Market                     69    
                      Average Days on Market YTD                  59   
                      Average Sale Price                     357,126   
                      Average Sale Price YTD                 374,397   
                      Listings                                1,238    
                      Listings YTD                            31,531   
                      Median Days on Market                      58    
                      Median Days on Market YTD                   43   
                      Median Sale Price                      340,000   
                      Median Sale Price YTD                  350,000   
                      New Listings                               NaN   
                      New Listings YTD                           NaN   
                      Sales                                     910    
                      Sales Volume                       324,984,560   
                      Sales Volume 

In [107]:

import pickle
filename = '../interim/dict_of_dfs_by_datetime.pkl'
with open(filename, 'wb') as f:
    pickle.dump(dfs, f)